DFA on DES, here we have 32 ciphers resulting from a fault injection on $R_{15}$

Helper functions

In [14]:
def is_nonzero(t):  #verifies a list contains non-zero elem
    for i in range(len(t)):
        if t[i] == 1:
            return True
    return False


def splitby4(t): #splits into a list of 4-uples
    i = 0
    ans = []
    while len(t) > len(ans)*4:        
        ans.append(t[i*4:(i+1)*4])
        i+= 1
    return ans

def int_to_bin(n): #binary rep of 64 bit int
    ans = []
    while(n > 0):
        ans.insert(0, n % 2) 
        n //= 2
    while(len(ans) < 64):
        ans.insert(0,0)
    return ans

def int_to_bin8(n): #binary rep of 8 bit int
    ans = []
    while(n > 0):
        ans.insert(0, n % 2)
        n //= 2
    while(len(ans) < 8):
        ans.insert(0,0)
    return ans



def int_to_bin6(n): #binary rep of 6 bit int
    ans = [] 
    while(n > 0):
        ans.insert(0, n % 2)
        n //= 2
    while(len(ans) < 6):
        ans.insert(0,0)
    return ans

def int_to_bin4(n): #binary rep of 4 bit int
    ans = []
    while(n > 0):
        ans.insert(0, n % 2)
        n //= 2
    while(len(ans) < 4):
        ans.insert(0,0)
    return ans

def bin_to_int(t): #binary to integer
    ans = 0
    while(len(t)>0):
        ans += 2**(len(t)-1)*t.pop(0)
    return ans

Feistel round internal functions

In [15]:
def Expansion(v):
    a =[32] + [i for i in range(1,6)] + [i for i in range(4,10)] + \
    [i for i in range(8,14)] + [i for i in range(12,18)] + \
    [i for i in range(16,22)] + [i for i in range(20,26)] + \
    [i for i in range(24,30)] + [i for i in range(28,33)] + [1]
    ans = []
    for i in range(48):
        ans.append(v[a[i]-1])
    return ans
    
def xor(v,w):
    ans = []
    for i in range(len(v)):
        ans.append((v[i] + w[i]) % 2)
    return ans

def Permutation(v):
    
    p = [16, 7, 20, 21, 29, 12, 28, 17, 1, 15, 23, 26, 5, 18, 31, 10,
         2, 8, 24, 14, 32, 27, 3, 9, 19, 13, 30, 6, 22, 11, 4, 25]
    ans = []
    
    for i in range(32):
        ans.append(v[p[i]-1])
    return ans



def Permutation_inv(v):
    
    p = [9, 17, 23, 31, 13, 28, 2, 18, 24, 16, 30, 6, 26, 20, 10, 1, 8,
         14, 25, 3, 4, 29, 11, 19, 32, 12, 22, 7, 5, 27, 15, 21]
    ans = []
    
    for i in range(32):
        ans.append(v[p[i]-1])
    return ans


SBOX = [
	[
		[14,4,13,1,2,15,11,8,3,10,6,12,5,9,0,7],
		[0,15,7,4,14,2,13,1,10,6,12,11,9,5,3,8],
		[4,1,14,8,13,6,2,11,15,12,9,7,3,10,5,0],
		[15,12,8,2,4,9,1,7,5,11,3,14,10,0,6,13]
	],
	[
		[15,1,8,14,6,11,3,4,9,7,2,13,12,0,5,10],
		[3,13,4,7,15,2,8,14,12,0,1,10,6,9,11,5],
		[0,14,7,11,10,4,13,1,5,8,12,6,9,3,2,15],
		[13,8,10,1,3,15,4,2,11,6,7,12,0,5,14,9]
	],
	[
		[10,0,9,14,6,3,15,5,1,13,12,7,11,4,2,8],
		[13,7,0,9,3,4,6,10,2,8,5,14,12,11,15,1],
		[13,6,4,9,8,15,3,0,11,1,2,12,5,10,14,7],
		[1,10,13,0,6,9,8,7,4,15,14,3,11,5,2,12]
	],
	[
		[7,13,14,3,0,6,9,10,1,2,8,5,11,12,4,15],
		[13,8,11,5,6,15,0,3,4,7,2,12,1,10,14,9],
		[10,6,9,0,12,11,7,13,15,1,3,14,5,2,8,4],
		[3,15,0,6,10,1,13,8,9,4,5,11,12,7,2,14]
	],
	[
		[2,12,4,1,7,10,11,6,8,5,3,15,13,0,14,9],
		[14,11,2,12,4,7,13,1,5,0,15,10,3,9,8,6],
		[4,2,1,11,10,13,7,8,15,9,12,5,6,3,0,14],
		[11,8,12,7,1,14,2,13,6,15,0,9,10,4,5,3]
	],
	[
		[12,1,10,15,9,2,6,8,0,13,3,4,14,7,5,11],
		[10,15,4,2,7,12,9,5,6,1,13,14,0,11,3,8],
		[9,14,15,5,2,8,12,3,7,0,4,10,1,13,11,6],
		[4,3,2,12,9,5,15,10,11,14,1,7,6,0,8,13]
	],
	[
		[4,11,2,14,15,0,8,13,3,12,9,7,5,10,6,1],
		[13,0,11,7,4,9,1,10,14,3,5,12,2,15,8,6],
		[1,4,11,13,12,3,7,14,10,15,6,8,0,5,9,2],
		[6,11,13,8,1,4,10,7,9,5,0,15,14,2,3,12]
	],
	[
		[13,2,8,4,6,15,11,1,10,9,3,14,5,0,12,7],
		[1,15,13,8,10,3,7,4,12,5,6,11,0,14,9,2],
		[7,11,4,1,9,12,14,2,0,6,10,13,15,3,5,8],
		[2,1,14,7,4,10,8,13,15,12,9,0,3,5,6,11]
	]
]

def sbox(index, B):
    
    r = 2*B[0] + B[5]
    c = bin_to_int(B[1:5])
    return int_to_bin4(SBOX[index-1][r][c])
    



the function $f$ , the initial permutation $IP$ and it's inverse $IP^{-1}$

In [16]:
def Feistel(R,K):
    R = Expansion(R)
    R = xor(R,K)
    R = sbox(1,R[0:6]) + sbox(2,R[6:12]) + sbox(3,R[12:18]) + sbox(4,R[18:24]) + \
    sbox(5,R[24:30]) + sbox(6,R[30:36]) + sbox(7,R[36:42]) + sbox(8,R[42:48])
    R = Permutation(R)
    return R

def IP(v):
    
    p = [58, 50, 42, 34, 26, 18, 10, 2, 60, 52, 44, 36, 28, 20, 12,
         4, 62, 54, 46, 38, 30, 22, 14, 6, 64, 56, 48, 40, 32, 24,
         16, 8, 57, 49, 41, 33, 25, 17, 9, 1, 59, 51, 43, 35, 27,19, 11,
         3, 61, 53, 45, 37, 29, 21, 13, 5, 63, 55, 47, 39, 31, 23, 15, 7]
    ans = []
    
    for i in range(64):
        ans.append(v[p[i]-1])
    return ans
    
def IP_inv(v):
    
    p = [40, 8, 48, 16, 56, 24, 64, 32, 39, 7, 47, 15, 55, 23, 63, 31,
         38, 6, 46, 14, 54, 22, 62, 30, 37, 5, 45, 13, 53, 21, 61, 29,
         36, 4, 44, 12, 52, 20, 60, 28, 35, 3, 43, 11, 51, 19, 59, 27,
         34, 2, 42, 10, 50, 18, 58, 26, 33, 1, 41, 9, 49, 17, 57, 25]
    ans = []
    
    for i in range(64):
        ans.append(v[p[i]-1])
    return ans
    

the function key combines all possible key slices

In [17]:
def key(K,j,K_j):
    ans = K
    if ans[j] == []:
        ans[j] = K_j
    else:
        ans[j] = collision(K[j],K_j)
    return ans                 

        
            
def collision(t_1,t_2): #extracts 2 lists' common elements
    ans = []         
    for i in t_1:      
        for j in t_2:
            if i == j:
                ans.append(i)
    return(ans)
    

We find the original 16th subkey $K_{16}$ from the plaintext, the ciphertext and 32 fault-induced ciphers of the same plaintext

In [23]:
clear = int_to_bin(int("6810C4AB6E8FA47F",16))

cipher = IP(int_to_bin(int("AA86867DED87275B",16)))

cipher_d = [
 int_to_bin(int("A887863DED86274B" ,16)),
 int_to_bin(int("AA94863DED87275B",16)),
 int_to_bin(int("AA968439ED86275B",16)),
 int_to_bin(int("AB96827FFD86275B",16)),
 int_to_bin(int("ABD6823DEF87275B",16)),
 int_to_bin(int("AAC6827DFD85275B",16)),
 int_to_bin(int("AB86827DED87255B",16)),
 int_to_bin(int("AB86867CBD832759",16)),
 int_to_bin(int("A2C6827CED93275B",16)),
 int_to_bin(int("AA8E867DAD93275B",16)),
 int_to_bin(int("AA868E7CED97275B",16)),
 int_to_bin(int("EA869674ADD7275B",16)),
 int_to_bin(int("AA86967CA593275A",16)),
 int_to_bin(int("AA86967DEDCF275A",16)),
 int_to_bin(int("EA86867DED872F5A",16)),
 int_to_bin(int("EA86967DED873652",16)),
 int_to_bin(int("8A86967DE9C7265B",16)),
 int_to_bin(int("AAA6867DE987261B",16)),
 int_to_bin(int("AA86A67DED87361B",16)),
 int_to_bin(int("BE86C75DE987361B",16)),
 int_to_bin(int("BE86C77DC987265B",16)),
 int_to_bin(int("BE86867DEDA7275B",16)),
 int_to_bin(int("AE86C67DED87075B",16)),
 int_to_bin(int("BA86C76DED87237B",16)),
 int_to_bin(int("3E86876DEC87635B",16)),
 int_to_bin(int("AA06866DEC87275B",16)),
 int_to_bin(int("AA86066DED87235B",16)),
 int_to_bin(int("AA8286EDED87635F",16)),
 int_to_bin(int("AA83866D6C87275B",16)),
 int_to_bin(int("AA87867DED07274B",16)),
 int_to_bin(int("AA82867DED87A75F",16)),
 int_to_bin(int("AA83863DED8627CB",16)) ]

R15 = cipher[32:64]
R16 = cipher[0:32]

K = [[],[],[],[],[],[],[],[]]
                    
for i in range(32):
    cipher_d[i] = IP(cipher_d[i]) 
    
    R15p = cipher_d[i][32:64] 
    R16p = cipher_d[i][0:32]
    
    dif = splitby4(Permutation_inv(xor(R16,R16p))) # coupe en groupe de 4 bits
    for j in range(8):
        K_j = []
        if is_nonzero(dif[j]): #si les groupe de 4 bits sont non nul alors on teste
            for k in range(2**6): # tout les 6 bits de clés possible qui
                it = int_to_bin6(k) # affecte la sbox
                Er15 = xor(Expansion(R15)[j*6:(j+1)*6],it)
                Er15p = xor(Expansion(R15p)[j*6:(j+1)*6],it)
                
                checkdif = xor(sbox(j+1,Er15) , sbox(j+1,Er15p))
                if checkdif == dif[j]:
                    K_j.append(it)
            K = key(K,j,K_j)
print(K[:4])
print(K[4:])

[[[1, 0, 1, 0, 1, 0]], [[1, 1, 0, 1, 0, 1]], [[1, 1, 0, 0, 0, 1]], [[0, 0, 1, 1, 1, 1]]]
[[[0, 0, 1, 1, 0, 0]], [[0, 1, 0, 0, 0, 1]], [[0, 0, 0, 0, 1, 0]], [[1, 1, 0, 1, 1, 0]]]


DES key schedule

In [24]:
def rotate(l, n):
    return l[n:] + l[:n]


def PC1(K):
    
    ans = [[],[]]
    
    C = [57, 49, 41, 33, 25, 17, 9, 1, 58, 50, 42, 34, 26, 18,
         10, 2, 59, 51, 43, 35, 27, 19, 11, 3, 60, 52, 44, 36]

    D = [63, 55, 47, 39, 31, 23, 15, 7, 62, 54, 46, 38, 30, 22,
         14, 6, 61, 53, 45, 37, 29, 21, 13, 5, 28, 20, 12, 4]
    
    for i in range(28):
        ans[0].append(K[C[i]-1])
        ans[1].append(K[D[i]-1])
    return ans


def PC2(K):
    
    
    p = [14, 17, 11, 24, 1, 5, 3, 28, 15, 6, 21, 10, 23, 19, 12, 4,
         26, 8, 16, 7, 27, 20, 13, 2, 41, 52, 31, 37, 47, 55, 30, 40,
         51, 45, 33, 48, 44, 49, 39, 56, 34, 53, 46, 42, 50, 36, 29, 32]
    ans = []
    
    for i in range(48):
        ans.append(K[p[i]-1])
    return ans
        
        
def DES_ks(K):
    K_i = []
    v = [1,1,2,2,2,2,2,2,1,2,2,2,2,2,2,1]
    T = PC1(K)
    for i in range(16):
        T[0] = rotate(T[0],v[i])
        T[1] = rotate(T[1],v[i])
        K_i.append(PC2(T[0]+T[1]))
    return K_i


this piece of code gives us where every bit from $K$ ends up in $K_{16}$

In [25]:
a = []
for i in range(64):
    a.append(i+1)
b = DES_ks(a)
print(b[15][:24])
print(b[15][24:])

[18, 59, 42, 3, 57, 25, 41, 36, 10, 17, 27, 50, 11, 43, 34, 33, 52, 1, 2, 9, 44, 35, 26, 49]
[30, 5, 47, 62, 45, 12, 55, 38, 13, 61, 31, 37, 6, 29, 46, 4, 23, 28, 53, 22, 21, 7, 63, 39]


wich helps us define KfromK16. the parity function just completes the key with the parity bit to get a 64 bit key

In [26]:
def KfromK16(K16): #returns one of the possible key such that the key schedule gives us K16
    K = [0]*64     #we default the unknown bits to 1
    
    p = [18, 59, 42, 3, 57, 25, 41, 36, 10, 17, 27, 50, 11, 43, 34,
         33, 52, 1, 2, 9, 44, 35, 26, 49,30, 5, 47, 62, 45, 12, 55,
         38, 13, 61, 31, 37, 6, 29, 46, 4, 23, 28, 53, 22, 21, 7, 63, 39]
    
    for i in range(48):
        K[p[i]-1] = K16[i]
    return K

def parity(T):
    
    for i in range(8):
        b = 0
        for j in range(7):
            b = (b + T[i*8 + j]) % 2
        T[((i+1)*8)-1] = (b+1) % 2
    return T
        

finally DES

In [27]:
def DES(M,K):
    K_i = DES_ks(K)
    (L,R) = IP(M)[0:32],IP(M)[32:64]
    
    for i in range(16):
        T = R
        R = xor(L,Feistel(R,K_i[i]))
        L = T
    return IP_inv(R+L)

And we end with an exhaustive search on the last bits

In [29]:
K = []

for i in range(2**8):
    
    K16 = [1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0,
           0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1,
           0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0]
    t = int_to_bin8(i)
    
    K0 = KfromK16(K16)
    
    K0[13] = t[0]
    K0[15] = t[1]
    K0[18] = t[2]
    K0[19] = t[3]
    K0[50] = t[4]
    K0[53] = t[5]
    K0[57] = t[6]
    K0[59] = t[7]
    
    K0 = parity(K0)
    
    if DES(int_to_bin(int("6810C4AB6E8FA47F",16)),K0) == \
    int_to_bin(int("AA86867DED87275B",16)):
        K = K0
print(K)


[1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0]


in hexadecimal $K$ = 83 20 C7 40 3D F2 C8 D6